In [1]:
import math
import torch
import numpy as np
import gpytorch
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

/home/cyy/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Using UMAP to map the features to a low dimensional space
1. loading data from MNIST dataset
2. choosing 600 data points from each training dataset (6000 in total)
3. choosing data points in propotion in the testing sets as well

In [2]:
# import umap.umap_ as umap
# from sklearn.datasets import fetch_openml
# import seaborn as sns
# from tensorflow import keras

2022-05-26 08:56:33.203792: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-26 08:56:33.204191: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# (x_tr, y_tr), (x_ts, y_ts) = keras.datasets.mnist.load_data()
# print(f'Train size: {len(y_tr)} \nTest size: {len(y_ts)}')

Train size: 60000 
Test size: 10000


In [4]:
# # train
# x_tr = x_tr[0:6000]
# x_tr = x_tr.reshape(-1, 784)
# reducer_tr = umap.UMAP(random_state = 42)
# embedding_tr = reducer_tr.fit_transform(x_tr) 

# # test
# x_ts = x_ts[0:1000]
# x_ts = x_ts.reshape(-1, 784)
# reducer_ts = umap.UMAP(random_state = 42)
# embedding_ts = reducer_ts.fit_transform(x_ts) 

# train_x, train_y, test_x, test_y = embedding_tr, y_tr[0:6000], embedding_ts, y_ts[0:1000]

In [2]:
# with open('features_UMAP.npy', 'wb') as f:
#     np.save(f, train_x)
#     np.save(f, train_y)
#     np.save(f, test_x)
#     np.save(f, test_y)


In [45]:
with open('features_UMAP.npy', 'rb') as f:
    train_x = np.load(f)
    train_y = np.load(f)
    test_x = np.load(f)
    test_y = np.load(f)
    


In [78]:
# with open('features_VAE.npy', 'rb') as f:
#     train_x = np.load(f)
#     train_y = np.load(f)
#     test_x = np.load(f)
#     test_y = np.load(f)

for binary training

In [6]:
# train_y01 = train_y[np.where(train_y <= 1)]
# train_x01 = train_x[np.where(train_y <= 1)]
# test_y01 = test_y[np.where(test_y <= 1)]
# test_x01 = test_x[np.where(test_y <= 1)]
# train_x, train_y, test_x, test_y = \
#     torch.from_numpy(train_x01), torch.from_numpy(train_y01), \
#         torch.from_numpy(test_x01), torch.from_numpy(test_y01)

using all classes

In [46]:
train_x, train_y, test_x, test_y = \
    torch.from_numpy(train_x), torch.from_numpy(train_y), \
        torch.from_numpy(test_x), torch.from_numpy(test_y)

In [4]:
train_y.shape

torch.Size([6000])

In [ ]:
import seaborn as sns
sns.set(context="paper", style="white")

fig, ax = plt.subplots(figsize=(12, 10))
color = train_y.astype(int)
plt.scatter(train_x[:, 0], train_x[:, 1], c=color, cmap="Spectral", s=0.1)
plt.setp(ax, xticks=[], yticks=[])
plt.gca().set_aspect('equal', 'datalim')
plt.title("MNIST data embedded into two dimensions by UMAP", fontsize=18)

plt.show()

## Setting up the Model

In [4]:
from gpytorch.models import ExactGP
from gpytorch.likelihoods import DirichletClassificationLikelihood
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel

In [61]:
train_x, train_y = train_x[0: 1000], train_y[0: 1000]

In [62]:
train_y = train_y.to(torch.int64)
test_y = test_y.to(torch.int64)

## Exact GP - One Model

In [55]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [ ]:
training_iter = 100

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    print(output.mean)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

In [57]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    observed_pred = likelihood(model(test_x))

In [58]:
pred_y = observed_pred.mean.round()

In [59]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred_y)

0.288

## Exact GP - Ten Models

In [64]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(train_x, train_y, likelihood)

In [65]:
models, likelihoods, mlls = [], [], []
for j in range(10):
    y = train_y
    x = train_x
    # y = train_y[train_y == j]
    # x = train_x[train_y == j]
    # y = torch.zeros_like(y)
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(train_x, train_y, likelihood)
    likelihood.train()
    model.train()
    likelihoods.append(likelihood)
    models.append(model)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    mlls.append(mll)

In [66]:
# import os
# smoke_test = ('CI' in os.environ)
training_iter = 100


# # Find optimal model hyperparameters
# model.train()
# likelihood.train()

# Use the adam optimizer
lr = 0.1
opts = []
for j in range(10):
    optimizer = torch.optim.Adam(models[j].parameters(), lr = lr)  # Includes GaussianLikelihood parameters
    opts.append(optimizer)

# # "Loss" for GPs - the marginal log likelihood
# mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

In [69]:
for j in range(10):
    print("Model", j)
    model = models[j]
    likelihood = likelihoods[j]
    mll = mlls[j]
    x = train_x #[train_y == j]
    optimizer = opts[j]
    # loss = losses[j]
    for i in range(training_iter):        
        
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(x)
        # print(output)
        
        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()
          
        if i % 1 == 0:
            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                i + 1, training_iter, loss.item(),
                model.covar_module.base_kernel.lengthscale.item(),
                model.likelihood.noise.item()
            ))
        optimizer.step()

Model 0
Iter 1/100 - Loss: 2.264   lengthscale: 0.693   noise: 0.693
Iter 2/100 - Loss: 2.147   lengthscale: 0.744   noise: 0.744
Iter 3/100 - Loss: 2.044   lengthscale: 0.797   noise: 0.798
Iter 4/100 - Loss: 1.971   lengthscale: 0.851   noise: 0.853
Iter 5/100 - Loss: 1.909   lengthscale: 0.905   noise: 0.909
Iter 6/100 - Loss: 1.862   lengthscale: 0.959   noise: 0.965
Iter 7/100 - Loss: 1.822   lengthscale: 1.012   noise: 1.021
Iter 8/100 - Loss: 1.800   lengthscale: 1.064   noise: 1.077
Iter 9/100 - Loss: 1.776   lengthscale: 1.113   noise: 1.131
Iter 10/100 - Loss: 1.762   lengthscale: 1.160   noise: 1.183
Iter 11/100 - Loss: 1.744   lengthscale: 1.205   noise: 1.232
Iter 12/100 - Loss: 1.732   lengthscale: 1.246   noise: 1.278
Iter 13/100 - Loss: 1.722   lengthscale: 1.284   noise: 1.322
Iter 14/100 - Loss: 1.728   lengthscale: 1.319   noise: 1.361
Iter 15/100 - Loss: 1.716   lengthscale: 1.351   noise: 1.396
Iter 16/100 - Loss: 1.719   lengthscale: 1.380   noise: 1.428
Iter 17/1

In [70]:
for model in models:
    model.eval()
for likelihood in likelihoods:
    likelihood.eval()

test_dists, preds = [], []
for j in range(10):
    with gpytorch.settings.fast_pred_var(), torch.no_grad():
        test_dist = models[j](test_x)
        pred_means = test_dist.loc
        test_dists.append(test_dist)
        preds.append(pred_means)

In [76]:
preds[0][1].round()

tensor(3.)

In [77]:
def most_common(lst):
    return max(set(lst), key=lst.count)

pred_y = []
for i in range(len(preds[0])):
    temp = []
    for j in range(10):
        temp.append(preds[j][i].round())
    pred_y.append(most_common(temp))
    
pred_y = torch.as_tensor(pred_y)

In [ ]:
pred_y

In [79]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred_y)

0.302

## One model

In [36]:
# We will use the simplest form of GP model, exact inference
class DirichletGPModel(ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_classes):
        super(DirichletGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean(batch_shape=torch.Size((num_classes,)))
        self.covar_module = ScaleKernel(
            RBFKernel(batch_shape=torch.Size((num_classes,))),
            batch_shape=torch.Size((num_classes,)),
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
# we let the DirichletClassificationLikelihood compute the targets for us
# print(train_y)
likelihood = DirichletClassificationLikelihood(train_y, learn_additional_noise = True)
model = DirichletGPModel(train_x, likelihood.transformed_targets, likelihood, num_classes = likelihood.num_classes)

Train and Fit the Model

In [37]:
# import os
# smoke_test = ('CI' in os.environ)
training_iter = 100


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
lr = 0.1
optimizer = torch.optim.Adam(model.parameters(), lr = lr)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

In [ ]:
for i in range(training_iter):
    # if(i%50 == 0):
    #     lr *= 0.1
    #     optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    print(likelihood(output).mean)
    # Calc loss and backprop gradients
    loss = -mll(output, likelihood.transformed_targets).sum()
    loss.backward()
    
    # for j in range(len(train_y)//100):
        
    #     # Zero gradients from previous iteration
    #     optimizer.zero_grad()
    #     # Output from model
    #     output = model(train_x[100*j: 100*(j+1)])
    #     # Calc loss and backprop gradients
    #     loss = -mll(output, likelihood.transformed_targets[:, 100*j: 100*(j+1)]).sum()
    #     loss.backward()
        
    if i % 1 == 0:
        print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
            i + 1, training_iter, loss.item(),
            model.covar_module.base_kernel.lengthscale.mean().item(),
            model.likelihood.second_noise_covar.noise.mean().item()
        ))
    optimizer.step()

Evaluation

In [26]:
# Put the model into evaluation mode
model.eval()
likelihood.eval()

# The gpytorch.settings.fast_pred_var flag activates LOVE (for fast variances)
# See https://arxiv.org/abs/1803.06058
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    # Obtain the predictive mean and covariance matrix
    f_preds = model(test_x)
    f_mean = f_preds.mean
    f_cov = f_preds.covariance_matrix
    
    # Make predictions by feeding model through likelihood
    observed_pred = likelihood(model(test_x))
    
    # # Initialize plot
    # f, ax = plt.subplots(1, 1, figsize=(8, 6))
    # # Get upper and lower confidence bounds
    # lower, upper = observed_pred.confidence_region()
    # # Plot training data as black stars
    # ax.plot(train_x.numpy(), train_y.numpy(), 'k*')
    # # Plot predictive means as blue line
    # ax.plot(test_x.numpy(), torch.transpose(observed_pred.mean, 0, 1).numpy(), 'b')
    # # Shade between the lower and upper confidence bounds
    # ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
    # # ax.set_ylim([-3, 3])
    # ax.legend(['Observed Data', 'Mean', 'Confidence'])

In [40]:
observed_pred.mean

tensor([[-6.7985, -6.9058, -6.9149,  ..., -7.3837, -6.9432, -6.8148],
        [-4.9146, -7.3742, -2.6759,  ..., -6.9776, -7.0634, -6.9719],
        [-6.0887, -3.5666, -3.0848,  ..., -6.6336, -7.4060, -6.8978],
        ...,
        [-2.0838, -6.1590, -2.8756,  ..., -6.1445, -6.8628, -6.5045],
        [-5.7520, -7.1136, -5.9357,  ..., -6.8909, -1.9709, -6.8109],
        [-2.6818, -5.8436, -6.5122,  ..., -5.2437, -6.4340, -5.7423]])

In [28]:
observed_pred.mean.shape

torch.Size([10, 1000])

In [29]:
pred_y = torch.argmax(observed_pred.mean, dim = 0)

In [95]:
model.eval()
likelihood.eval()


with gpytorch.settings.fast_pred_var(), torch.no_grad():
    test_dist = model(test_x)
    pred_means = test_dist.loc

In [96]:
pred_y = torch.argmax(pred_means, dim = 0)

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred_y)

0.439

## Ten models

model structure

In [7]:
class DirichletGPModel(ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_classes):
        super(DirichletGPModel, self).__init__(train_x, train_y, likelihood)
        # num_classes = 1
        self.mean_module = ConstantMean(batch_shape=torch.Size((num_classes,)))
        self.covar_module = ScaleKernel(
            RBFKernel(batch_shape=torch.Size((num_classes,))),
            batch_shape=torch.Size((num_classes,)),
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

model initialization

In [8]:
models, likelihoods, mlls = [], [], []
for j in range(10):
    y = train_y
    x = train_x
    # y = train_y[train_y == j]
    # x = train_x[train_y == j]
    # y = torch.zeros_like(y)
    likelihood = DirichletClassificationLikelihood(y, learn_additional_noise = True)
    model = DirichletGPModel(x, likelihood.transformed_targets, likelihood, num_classes = likelihood.num_classes)
    likelihood.train()
    model.train()
    likelihoods.append(likelihood)
    models.append(model)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    mlls.append(mll)

method 1

In [63]:
# import os
# smoke_test = ('CI' in os.environ)
training_iter = 100


# # Find optimal model hyperparameters
# model.train()
# likelihood.train()

# Use the adam optimizer
lr = 0.1

optimizer = torch.optim.Adam(models[j].parameters(), lr = lr)  # Includes GaussianLikelihood parameters

# # "Loss" for GPs - the marginal log likelihood
# mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

In [14]:
train_x.shape

torch.Size([1000, 2])

In [ ]:
for i in range(training_iter):              
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    outputs, all_loss = [], []
    for j in range(10):
        model = models[j]
        likelihood = likelihoods[j]
        output = likelihood(model(train_x))
        output2 = model(train_x)
        print(output.loc.shape, output2.loc.shape)
        print(output.loc == output2.loc)
        # output = torch.transpose(output.mean, 0, 1)
        # outputs.append(output.loc)
        # print(train_y.shape, output.mean.shape)
        loss_ = torch.nn.CrossEntropyLoss()
        loss = loss_(output, train_y)
        all_loss.append(loss)
    # print(output)
        
    # Calc loss and backprop gradient  
    loss = torch.as_tensor(loss)
    loss = torch.mean(loss)  
    loss.backward()
          
    if i % 1 == 0:
        print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
            i + 1, training_iter, loss.item(),
            model.covar_module.base_kernel.lengthscale.mean().item(),
            model.likelihood.second_noise_covar.noise.mean().item()
        ))
    optimizer.step()

method 2

In [ ]:
# import os
# smoke_test = ('CI' in os.environ)
training_iter = 100


# # Find optimal model hyperparameters
# model.train()
# likelihood.train()

# Use the adam optimizer
lr = 0.1
opts = []
for j in range(10):
    optimizer = torch.optim.Adam(models[j].parameters(), lr = lr)  # Includes GaussianLikelihood parameters
    opts.append(optimizer)

# # "Loss" for GPs - the marginal log likelihood
# mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

In [114]:
for j in range(10):
    print("Model", j)
    model = models[j]
    likelihood = likelihoods[j]
    mll = mlls[j]
    x = train_x #[train_y == j]
    optimizer = opts[j]
    # loss = losses[j]
    for i in range(training_iter):        
        
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(x)
        # print(output)
        
        # Calc loss and backprop gradients
        loss = -mll(output, likelihood.transformed_targets).sum()
        loss.backward()
          
        if i % 1 == 0:
            print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                i + 1, training_iter, loss.item(),
                model.covar_module.base_kernel.lengthscale.mean().item(),
                model.likelihood.second_noise_covar.noise.mean().item()
            ))
        optimizer.step()

Model 0
Iter 1/100 - Loss: 25.757   lengthscale: 0.693   noise: 0.693
Iter 2/100 - Loss: 24.706   lengthscale: 0.744   noise: 0.644
Iter 3/100 - Loss: 23.764   lengthscale: 0.798   noise: 0.598
Iter 4/100 - Loss: 22.971   lengthscale: 0.853   noise: 0.555
Iter 5/100 - Loss: 22.356   lengthscale: 0.909   noise: 0.515
Iter 6/100 - Loss: 21.855   lengthscale: 0.966   noise: 0.480
Iter 7/100 - Loss: 21.351   lengthscale: 1.023   noise: 0.449
Iter 8/100 - Loss: 21.052   lengthscale: 1.080   noise: 0.422
Iter 9/100 - Loss: 20.713   lengthscale: 1.137   noise: 0.400
Iter 10/100 - Loss: 20.469   lengthscale: 1.193   noise: 0.380
Iter 11/100 - Loss: 20.242   lengthscale: 1.247   noise: 0.364
Iter 12/100 - Loss: 20.097   lengthscale: 1.300   noise: 0.350
Iter 13/100 - Loss: 19.855   lengthscale: 1.352   noise: 0.339
Iter 14/100 - Loss: 19.773   lengthscale: 1.401   noise: 0.330
Iter 15/100 - Loss: 19.634   lengthscale: 1.449   noise: 0.322
Iter 16/100 - Loss: 19.488   lengthscale: 1.495   noise:

## Model Predictions

In [115]:
for model in models:
    model.eval()
for likelihood in likelihoods:
    likelihood.eval()

test_dists, preds = [], []
for j in range(10):
    with gpytorch.settings.fast_pred_var(), torch.no_grad():
        test_dist = models[j](test_x)
        pred_means = test_dist.loc
        test_dists.append(test_dist)
        preds.append(pred_means)

In [129]:
len(preds[0][0])

1000

In [8]:
# preds_all = torch.zeros(10, len(preds[0][0]))
# for i in range(10):
#     preds_all += preds[i]
# preds_all.shape

# pred_y = torch.argmax(preds_all, dim = 0)

In [126]:
preds_y = []
for i in range(10):
    pred_y = torch.argmax(preds[i], dim = 0)
    preds_y.append(pred_y)

In [128]:
len(preds_y)

10

In [130]:
def most_common(lst):
    return max(set(lst), key=lst.count)

pred_y = []
for i in range(len(preds[0][0])):
    temp = []
    for j in range(10):
        temp.append(preds_y[j][i])
    pred_y.append(most_common(temp))
    
pred_y = np.array(pred_y)
pred_y = torch.as_tensor(pred_y)

In [98]:
# preds[0][0, 1], preds[1][0, 1], preds[5][0, 1], preds[6][0, 1], preds[7][0, 1], preds[8][0, 1], preds[9][0, 1]

In [124]:
pred_y[0:5]

tensor([7, 3, 1, 5, 4])

In [102]:
test_y[0:10]

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9])

In [135]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred_y)

0.439